[RNN: You've been reading it the wrong way](https://www.kaggle.com/code/deshwalmahesh/rnn-you-ve-been-reading-it-the-wrong-way/notebook)


  1) RNN works in Parallel

  2) There is just one Weight matrix which is important

  3) Number of neurons is "How many time steps to create"

  4) Hidden state is a number



**RNN**

In [7]:
import torch

In [29]:
#'input_size' and 'hidden_size'
#input_size – The number of expected features in the input x
#hidden_size – The number of features in the hidden state h

#N=4, Hin=100, Hout=10
x = torch.ones((4, 100))
h_0 = torch.ones((1, 10))
rnn_ceil = torch.nn.RNN(100, 10, num_layers=1)
result = rnn_ceil(x, h_0)
print(result[0].shape, result[1].shape)

torch.Size([4, 10]) torch.Size([1, 10])


In [ ]:
# h_t[] = x_t[4x100]*Wi[100x10] + bi[10] + h_t-1[1x10]*Wh[10x10] + bh[10]

In [26]:
for parameter in rnn_ceil.all_weights[0]:
  print(parameter.shape)

torch.Size([10, 100])
torch.Size([10, 10])
torch.Size([10])
torch.Size([10])


**Paper:** Zhao, Tianqi. "Deep multimodal learning: An effective method for video classification." 2019 IEEE International Conference on Web Services (ICWS). IEEE, 2019.

**Dataset:** https://research.google.com/youtube8m/

In [4]:
#Задача 1: реализуйте GAP метрику из статьи
#https://en.wikipedia.org/wiki/Precision_and_recall
def global_average_precision_score(y_true, y_pred):
    pass

In [5]:
y_true = {
        'id_001': 123,
        'id_002': None,
        'id_003': 999,
        'id_004': 123,
        'id_005': 999,
        'id_006': 888,
        'id_007': 666,
        'id_008': 666,
        'id_009': None,
        'id_010': 666,
    }

y_pred = {
        'id_001': (123, 0.15),
        'id_002': (123, 0.10),
        'id_003': (999, 0.30),
        'id_005': (999, 0.40),
        'id_007': (555, 0.60),
        'id_008': (666, 0.70),
        'id_010': (666, 0.99),
    }

In [ ]:
assert round(global_average_precision_score(y_true, y_pred), 3) == 0.548

**Pytorch and video**

https://pytorchvideo.org/
https://github.com/facebookresearch/pytorchvideo

In [34]:
sample = torch.ones((2, 10, 3, 5, 5))

In [44]:
sample[:, 0].shape

torch.Size([2, 3, 5, 5])

In [46]:
baseModel = models.resnet18(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [54]:
baseModel

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [92]:
from torch import nn
from torchvision import models
from torchvision.models.feature_extraction import create_feature_extractor

class Resnt18Rnn(nn.Module):
    def __init__(self):
        super().__init__()
        num_classes = 10
        rnn_hidden_size = 100
        rnn_num_layers = 1
        
        model = models.resnet18(pretrained=True)
        num_features = model.fc.in_features
        self.baseModel = create_feature_extractor(model, {"avgpool": "features"})

        self.dropout= nn.Dropout()
        self.rnn = nn.RNN(num_features, rnn_hidden_size, rnn_num_layers, batch_first=True)
        self.fc1 = nn.Linear(rnn_hidden_size, num_classes)

    def forward(self, x):
        b, n, c, h, w = x.shape
        i = 0
        y = self.baseModel((x[:, i])).get("features").flatten(1)
        y = y.unsqueeze(1) #подаем по одному
        print(y.shape)
        output, hn = self.rnn(y)
        print(output.shape)
        print(hn.shape)

        for i in range(1, n):
            y = self.baseModel((x[:, i])).get("features").flatten(1)
            out, hn = self.rnn(y.unsqueeze(1), hn)
    
        out = self.dropout(out[:,-1])
        print(out.shape)
        out = self.fc1(out) 
        return out 

In [93]:
simpleModel = Resnt18Rnn()

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [94]:
simpleModel(sample).shape

torch.Size([2, 1, 512])
torch.Size([2, 1, 100])
torch.Size([1, 2, 100])
torch.Size([2, 100])


torch.Size([2, 10])